# Data Augmentation

As demonstrated in the EDA notebook of this project, there are marked differences between wheat images across domains. The original challenge pre-divided the image dataset into train, test, and validation folders, with *completely distinct* sets of domains in each folder. Even if the images were apparently very similar across domains, this would not be a wise project design. But we've also demonstrated that there are clear differences between images from varying domains. To correct for this, we will combine the pre-divided train, test, and validation folders, augment the full set of data to account for any class imbalances, and then re-split into testing, training, and validation folders with even (or, as even as possible) representation across image sets. 

In [42]:
import pandas as pd
import pathlib

In [3]:
test = pd.read_csv('gwhd_2021/competition_test.csv')
train = pd.read_csv('gwhd_2021/competition_train.csv')
val = pd.read_csv('gwhd_2021/competition_val.csv')

In [7]:
full = pd.concat([train, test, val])

We'll save this full dataframe to a csv file:

In [12]:
# full.to_csv('transformed_data/train_test_val_concat.csv')

Now, we'll need to perform the same basic transformations to this new dataframe, as we did to the original training dataset dataframe (like reformatting of bounding box coordinates, etc). We'll also want to merge the metadata information so that we can address class imbalances when performing data augmentation and when performing the new splits. 

In [15]:
metadata = pd.read_csv('gwhd_2021/metadata_dataset.csv', sep=';')

In [18]:
# Update metadata['name'] to metadata['domain'] to match full column name
metadata.rename(columns = {"name": "domain"}, inplace=True)

In [22]:
df = full.merge(metadata, on='domain')

Change image name to read `.jpg` instead of `.png`:

In [28]:
for i, row in df.iterrows():
    row['image_name'] = (row.image_name.split('.')[0]+'.jpg')

Check for duplicated images:

In [32]:
df.image_name[df.image_name.duplicated()]

2070    d88963636d49127bda0597ef73f1703e92d6f111caefc4...
2079    1961bcf453d5b2206c428c1c14fe55d1f26f3c655db0a2...
3948    da9846512ff19b8cd7278c8c973f75d36de8c4eb4e593b...
Name: image_name, dtype: object

In [35]:
df[df.image_name == df.iloc[2070, 0]]

,image_name,BoxesString,domain,country,location,development_stage
1986,d88963636d49127bda0597ef73f1703e92d6f111caefc4...,994 8 1024 64;864 0 1012 92;642 18 772 102;616...,Arvalis_10,France,Mons,Filling
2070,d88963636d49127bda0597ef73f1703e92d6f111caefc4...,312 994 364 1024;177 756 296 869;88 118 147 17...,Arvalis_12,France,Gréoux,Filling


In [37]:
df[df.image_name == df.iloc[2079, 0]]

,image_name,BoxesString,domain,country,location,development_stage
1999,1961bcf453d5b2206c428c1c14fe55d1f26f3c655db0a2...,264 164 370 296;388 196 464 330;466 328 698 44...,Arvalis_10,France,Mons,Filling
2079,1961bcf453d5b2206c428c1c14fe55d1f26f3c655db0a2...,3 944 58 985;876 506 919 555;770 859 834 903;9...,Arvalis_12,France,Gréoux,Filling


In [40]:
df[df.image_name == df.iloc[3948, 0]]

,image_name,BoxesString,domain,country,location,development_stage
3821,da9846512ff19b8cd7278c8c973f75d36de8c4eb4e593b...,950 39 986 165;725 2 776 226;660 31 720 92;797...,CIMMYT_1,Mexico,Ciudad Obregon,Post-flowering
3948,da9846512ff19b8cd7278c8c973f75d36de8c4eb4e593b...,733 0 837 49;688 96 998 196;792 433 843 461;73...,CIMMYT_3,Mexico,Ciudad Obregon,Post-flowering


Once we define functions to display images with bounding boxes, we'll need to make sure to check out the duplicates detected above. 

In [43]:
#create data directory path and count images
data_dir = pathlib.Path('gwhd_2021/images')

In [44]:
images = list(data_dir.glob('*.jpg'))